In [1]:
from skimage.feature import local_binary_pattern
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

def calcula_lbp(imagem, P, R, metodo):
    # Calcular o LBP
    lbp = local_binary_pattern(imagem, P, R, metodo)
    # Calcular o histograma LBP
    n_bins = int(lbp.max() + 1)
    hist, bins = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)
    return hist

# Abrir o arquivo para escrita
arquivo_resultados = open('resultados.txt', 'w')

# Parâmetros LBP
P = 8  # Número de pontos vizinhos
R = 2  # Raio
metodo = 'nri_uniform'

# Diretórios
diretorios = ['base/fold1', 'base/fold2', 'base/fold3']

hist = []
tags = []

# Carregar as imagens e calcular os histogramas LBP
for i, diretorio in enumerate(diretorios):
    lista_arquivos = os.listdir(diretorio)
    for j, arquivo_img in enumerate(lista_arquivos):
        caminho_imagem = os.path.join(diretorio, arquivo_img)
        imagem = cv2.imread(caminho_imagem, cv2.IMREAD_GRAYSCALE)
        hist.append(calcula_lbp(imagem, P, R, metodo))
        tags.append(j // 30 + 1)  # Tag para cada imagem

hist_ori = np.array(hist)
tags_ori = np.array(tags)

In [2]:
from skimage.feature import local_binary_pattern
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

def calcula_lbp(imagem, P, R, metodo):
    # Calcular o LBP
    lbp = local_binary_pattern(imagem, P, R, metodo)
    # Calcular o histograma LBP
    n_bins = int(lbp.max() + 1)
    hist, bins = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)
    return hist

# Abrir o arquivo para escrita
arquivo_resultados = open('resultados.txt', 'w')

# Parâmetros LBP
P = 8  # Número de pontos vizinhos
R = 2  # Raio
metodo = 'nri_uniform'

# Diretórios
diretorios = ['harmonico/fold1', 'harmonico/fold2', 'harmonico/fold3']

hist = []
tags = []

# Carregar as imagens e calcular os histogramas LBP
for i, diretorio in enumerate(diretorios):
    lista_arquivos = os.listdir(diretorio)
    for j, arquivo_img in enumerate(lista_arquivos):
        caminho_imagem = os.path.join(diretorio, arquivo_img)
        imagem = cv2.imread(caminho_imagem, cv2.IMREAD_GRAYSCALE)
        hist.append(calcula_lbp(imagem, P, R, metodo))
        tags.append(j // 30 + 1)  # Tag para cada imagem

hist_har = np.array(hist)
tags_har = np.array(tags)

In [3]:
from skimage.feature import local_binary_pattern
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

def calcula_lbp(imagem, P, R, metodo):
    # Calcular o LBP
    lbp = local_binary_pattern(imagem, P, R, metodo)
    # Calcular o histograma LBP
    n_bins = int(lbp.max() + 1)
    hist, bins = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)
    return hist

# Abrir o arquivo para escrita
arquivo_resultados = open('resultados.txt', 'w')

# Parâmetros LBP
P = 8  # Número de pontos vizinhos
R = 2  # Raio
metodo = 'nri_uniform'

# Diretórios
diretorios = ['percussivo/fold1', 'percussivo/fold2', 'percussivo/fold3']

hist = []
tags = []

# Carregar as imagens e calcular os histogramas LBP
for i, diretorio in enumerate(diretorios):
    lista_arquivos = os.listdir(diretorio)
    for j, arquivo_img in enumerate(lista_arquivos):
        caminho_imagem = os.path.join(diretorio, arquivo_img)
        imagem = cv2.imread(caminho_imagem, cv2.IMREAD_GRAYSCALE)
        hist.append(calcula_lbp(imagem, P, R, metodo))
        tags.append(j // 30 + 1)  # Tag para cada imagem

hist_per = np.array(hist)
tags_per = np.array(tags)

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Função para realizar a validação cruzada
def kfoldcv(hist, tags, k):
    fold_size = len(hist) // k
    accuracies = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Configurar os parâmetros para o GridSearchCV (SVM)
        param_grid = {
            'C': [0.289],
            'gamma': [1],
            'kernel': ['linear', 'rbf']
        }

        svm_model = SVC(random_state=42)
        grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        
        # Melhor modelo encontrado pelo Grid Search
        best_model = grid_search.best_estimator_

        # Fazer previsões e calcular a acurácia
        y_pred = best_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    mean_accuracy_ori = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy_ori = np.std(accuracies) * 100  # Converter para porcentagem

    return mean_accuracy_ori, std_accuracy_ori

# Realizar a validação cruzada
mean_accuracy_ori, std_accuracy_ori = kfoldcv(hist_ori, tags_ori, 3)

print(f'Acurácia média: {mean_accuracy_ori:.2f}%')
print(f'Desvio padrão da acurácia: {std_accuracy_ori:.2f}%')


Acurácia média: 79.22%
Desvio padrão da acurácia: 1.66%


In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Função para realizar a validação cruzada
def kfoldcv(hist, tags, k):
    fold_size = len(hist) // k
    accuracies = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Configurar os parâmetros para o GridSearchCV (SVM)
        param_grid = {
            'C': [0.289],
            'gamma': [1],
            'kernel': ['linear', 'rbf']
        }

        svm_model = SVC(random_state=42)
        grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        
        # Melhor modelo encontrado pelo Grid Search
        best_model = grid_search.best_estimator_

        # Fazer previsões e calcular a acurácia
        y_pred = best_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    mean_accuracy_har = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy_har = np.std(accuracies) * 100  # Converter para porcentagem

    return mean_accuracy_har, std_accuracy_har

# Realizar a validação cruzada
mean_accuracy_har, std_accuracy_har = kfoldcv(hist_har, tags_har, 3)

print(f'Acurácia média: {mean_accuracy_har:.2f}%')
print(f'Desvio padrão da acurácia: {std_accuracy_har:.2f}%')


Acurácia média: 74.44%
Desvio padrão da acurácia: 3.50%


In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Função para realizar a validação cruzada
def kfoldcv(hist, tags, k):
    fold_size = len(hist) // k
    accuracies = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Configurar os parâmetros para o GridSearchCV (SVM)
        param_grid = {
            'C': [0.289],
            'gamma': [1],
            'kernel': ['linear', 'rbf']
        }

        svm_model = SVC(random_state=42)
        grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        
        # Melhor modelo encontrado pelo Grid Search
        best_model = grid_search.best_estimator_

        # Fazer previsões e calcular a acurácia
        y_pred = best_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    mean_accuracy = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy = np.std(accuracies) * 100  # Converter para porcentagem

    return mean_accuracy, std_accuracy

# Realizar a validação cruzada
mean_accuracy_per, std_accuracy_per = kfoldcv(hist_per, tags_per, 3)

print(f'Acurácia média: {mean_accuracy_per:.2f}%')
print(f'Desvio padrão da acurácia: {std_accuracy_per:.2f}%')


Acurácia média: 75.33%
Desvio padrão da acurácia: 2.37%


In [8]:
# Função para realizar a validação cruzada
def kfoldcv(hist, tags, k):
    fold_size = len(hist) // k
    accuracies = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Configurar os parâmetros para o GridSearchCV (SVM)
        param_grid = {
            'C': [50, 10, 1, 0.1, 0.001, 0.0001],
            'gamma': [100, 50, 10, 1, 0.1, 0.50, 0.001, 0.0001],
            'kernel': ['rbf']
        }

        svm_model = SVC(random_state=42)
        grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        
        # Melhor modelo encontrado pelo Grid Search
        best_model = grid_search.best_estimator_

        # Fazer previsões e calcular a acurácia
        y_pred = best_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    mean_accuracy = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy = np.std(accuracies) * 100  # Converter para porcentagem

    return mean_accuracy, std_accuracy

# Realizar a validação cruzada
mean_accuracy, std_accuracy = kfoldcv(hist, tags, 3)

print(f'Acurácia média: {mean_accuracy:.2f}%')
print(f'Desvio padrão da acurácia: {std_accuracy:.2f}%')


Acurácia média: 73.44%
Desvio padrão da acurácia: 0.31%


In [7]:
from sklearn.ensemble import RandomForestClassifier

# Função para realizar a validação cruzada
def kfoldcv(hist, tags, k):
    # Assumindo que os dados já estão ordenados conforme os folds nos diretórios
    fold_size = len(hist) // k
    
    accuracies = []
    
    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        # Treinar o modelo SVM
        rf_model = RandomForestClassifier(n_estimators=4000, random_state=0, n_jobs=-1)
        rf_model.fit(X_train, y_train)
         
        # Fazer previsões e calcular a acurácia
        y_pred = rf_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    mean_accuracy = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy = np.std(accuracies) * 100  # Converter para porcentagem

    return mean_accuracy, std_accuracy

# Realizar a validação cruzada
mean_accuracy, std_accuracy = kfoldcv(hist, tags, 3)

print(f'Acurácia média: {mean_accuracy:.2f}')
print(f'Desvio padrão da acurácia: {std_accuracy:.2f}')

Acurácia média: 66.00
Desvio padrão da acurácia: 1.89


In [12]:
# Função para realizar a validação cruzada
def kfoldcv(hist, tags, k):
    fold_size = len(hist) // k
    accuracies = []
    
    for i in range(k):
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        param_grid = {
            'n_estimators': [200, 2000, 4000],
            'max_depth': [None, 10, 20, 30, 40, 50]
        }
        rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
        grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    mean_accuracy = np.mean(accuracies) * 100
    std_accuracy = np.std(accuracies) * 100

    return mean_accuracy, std_accuracy


# Realizar a validação cruzada
mean_accuracy, std_accuracy = kfoldcv(hist, tags, 3)

print(f'Acurácia média: {mean_accuracy:.2f}')
print(f'Desvio padrão da acurácia: {std_accuracy:.2f}')

Acurácia média: 66.11
Desvio padrão da acurácia: 1.97


In [13]:
from sklearn.neighbors import KNeighborsClassifier

# Função para realizar a validação cruzada com KNN
def kfoldcv_knn(hist, tags, k):
    # Assumindo que os dados já estão ordenados conforme os folds nos diretórios
    fold_size = len(hist) // k
    accuracies = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])

        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]

        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Definir a grade de parâmetros para o Grid Search
        param_grid = {'n_neighbors': np.arange(1, 21)}

        # Treinar o modelo KNN usando Grid Search
        knn_model = KNeighborsClassifier()
        grid_search = GridSearchCV(knn_model, param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)

        # Melhor modelo encontrado pelo Grid Search
        best_knn = grid_search.best_estimator_

        # Fazer previsões e calcular a acurácia
        y_pred = best_knn.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

    mean_accuracy = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy = np.std(accuracies) * 100  # Converter para porcentagem

    return mean_accuracy, std_accuracy

# Realizar a validação cruzada com KNN
mean_accuracy, std_accuracy = kfoldcv_knn(hist, tags, 3)

print(f'Acurácia média: {mean_accuracy:.2f}%')
print(f'Desvio padrão da acurácia: {std_accuracy:.2f}%')

Acurácia média: 64.33%
Desvio padrão da acurácia: 0.72%


In [45]:
from sklearn.metrics import accuracy_score
import numpy as np

# Função para realizar predições com probabilidades
def kfoldcv_proba(hist, tags, k):
    fold_size = len(hist) // k
    all_probas = []
    y_tests = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Configurar os parâmetros para o GridSearchCV (SVM)
        param_grid = {
            'C': [0.289],
            'gamma': [1],
            'kernel': ['linear', 'rbf']
        }

        svm_model = SVC(random_state=42, probability=True)
        grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        
        # Melhor modelo encontrado pelo Grid Search
        best_model = grid_search.best_estimator_

        # Fazer previsões de probabilidades
        y_proba = best_model.predict_proba(X_test)
        all_probas.append(y_proba)
        y_tests.append(y_test)
    
    return np.concatenate(all_probas), np.concatenate(y_tests)

# Realizar a validação cruzada com probabilidades para cada conjunto de dados
proba_ori, y_test = kfoldcv_proba(hist_ori, tags_ori, 3)
proba_har, _ = kfoldcv_proba(hist_har, tags_har, 3)
proba_per, _ = kfoldcv_proba(hist_per, tags_per, 3)

# Fusão das predições usando a regra da soma
fused_proba_soma = (proba_ori + proba_har + proba_per) / 3
fused_pred_soma = np.argmax(fused_proba_soma, axis=1)

for i in range(0, len(fused_pred_soma)):
    fused_pred_soma[i] = fused_pred_soma[i] + 1

# Fusão das predições usando a regra do produto
fused_proba_produto = proba_ori * proba_har * proba_per
fused_pred_produto = np.argmax(fused_proba_produto, axis=1)

for i in range(0, len(fused_pred_produto)):
    fused_pred_produto[i] = fused_pred_produto[i] + 1

# Avaliação das fusões
accuracy_soma = accuracy_score(y_test, fused_pred_soma)
accuracy_produto = accuracy_score(y_test, fused_pred_produto)

print(f'Acurácia (regra da soma): {accuracy_soma * 100:.2f}%')
print(f'Acurácia (regra do produto): {accuracy_produto * 100:.2f}%')


Acurácia (regra da soma): 81.00%
Acurácia (regra do produto): 81.56%


In [46]:
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Função para realizar predições com probabilidades usando Random Forest
def kfoldcv_proba_rf(hist, tags, k):
    fold_size = len(hist) // k
    all_probas = []
    y_tests = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Configurar os parâmetros para o GridSearchCV (Random Forest)
        param_grid = {
            'n_estimators': [1000, 2000, 4000],
            'max_features': ['sqrt', 'log2'],
            'max_depth': [None, 10, 20],
            'random_state': [42]
        }

        rf_model = RandomForestClassifier(n_jobs=-1)
        grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        
        # Melhor modelo encontrado pelo Grid Search
        best_model = grid_search.best_estimator_

        # Fazer previsões de probabilidades
        y_proba = best_model.predict_proba(X_test)
        all_probas.append(y_proba)
        y_tests.append(y_test)
    
    return np.concatenate(all_probas), np.concatenate(y_tests)

# Realizar a validação cruzada com probabilidades para cada conjunto de dados
proba_ori, y_test = kfoldcv_proba_rf(hist_ori, tags_ori, 3)
proba_har, _ = kfoldcv_proba_rf(hist_har, tags_har, 3)
proba_per, _ = kfoldcv_proba_rf(hist_per, tags_per, 3)

# Fusão das predições usando a regra da soma
fused_proba_soma = (proba_ori + proba_har + proba_per) / 3
fused_pred_soma = np.argmax(fused_proba_soma, axis=1)

# Incrementar as predições para começar de 1, conforme necessário
fused_pred_soma = fused_pred_soma + 1

# Fusão das predições usando a regra do produto
fused_proba_produto = proba_ori * proba_har * proba_per
fused_pred_produto = np.argmax(fused_proba_produto, axis=1)

# Incrementar as predições para começar de 1, conforme necessário
fused_pred_produto = fused_pred_produto + 1

# Avaliação das fusões
accuracy_soma = accuracy_score(y_test, fused_pred_soma)
accuracy_produto = accuracy_score(y_test, fused_pred_produto)

print(f'Acurácia (regra da soma): {accuracy_soma * 100:.2f}%')
print(f'Acurácia (regra do produto): {accuracy_produto * 100:.2f}%')


Acurácia (regra da soma): 71.44%
Acurácia (regra do produto): 72.44%


In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.model_selection import GridSearchCV

# Função para realizar predições com probabilidades
def kfoldcv_proba(hist, tags, k):
    fold_size = len(hist) // k
    all_probas = []
    y_tests = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Configurar os parâmetros para o GridSearchCV (SVM)
        param_grid = {
            'C': [0.289],
            'gamma': [1],
            'kernel': ['linear', 'rbf']
        }

        svm_model = SVC(random_state=42, probability=True)
        grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        
        # Melhor modelo encontrado pelo Grid Search
        best_model = grid_search.best_estimator_

        # Fazer previsões de probabilidades
        y_proba = best_model.predict_proba(X_test)
        all_probas.append(y_proba)
        y_tests.append(y_test)
    
    return np.concatenate(all_probas), np.concatenate(y_tests)

# Realizar a validação cruzada com probabilidades para cada conjunto de dados
proba_ori_svm, y_test = kfoldcv_proba(hist_ori, tags_ori, 3)
proba_har_svm, _ = kfoldcv_proba(hist_har, tags_har, 3)
proba_per_svm, _ = kfoldcv_proba(hist_per, tags_per, 3)

# Fusão das predições usando a regra da soma
fused_proba_soma = (proba_ori_svm + proba_har_svm + proba_per_svm) / 3
fused_pred_soma = np.argmax(fused_proba_soma, axis=1)

for i in range(0, len(fused_pred_soma)):
    fused_pred_soma[i] = fused_pred_soma[i] + 1

# Fusão das predições usando a regra do produto
fused_proba_produto = proba_ori_svm * proba_har_svm * proba_per_svm
fused_pred_produto = np.argmax(fused_proba_produto, axis=1)

for i in range(0, len(fused_pred_produto)):
    fused_pred_produto[i] = fused_pred_produto[i] + 1

# Avaliação das fusões

def evaluate_predictions(y_true, y_pred, method_name):
    accuracy = accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred, average='weighted') * 100
    recall = recall_score(y_true, y_pred, average='weighted') * 100
    f1 = f1_score(y_true, y_pred, average='weighted') * 100

    print(f'{method_name} - Acurácia: {accuracy:.2f}%')
    print(f'{method_name} - Precisão: {precision:.2f}%')
    print(f'{method_name} - Recall: {recall:.2f}%')
    print(f'{method_name} - F1-score: {f1:.2f}%')

# Avaliar as predições usando a regra da soma
evaluate_predictions(y_test, fused_pred_soma, "Regra da Soma")

print('----------------------------------------------------')

# Avaliar as predições usando a regra do produto
evaluate_predictions(y_test, fused_pred_produto, "Regra do Produto")


Regra da Soma - Acurácia: 81.00%
Regra da Soma - Precisão: 81.20%
Regra da Soma - Recall: 81.00%
Regra da Soma - F1-score: 80.99%
----------------------------------------------------
Regra do Produto - Acurácia: 81.56%
Regra do Produto - Precisão: 81.79%
Regra do Produto - Recall: 81.56%
Regra do Produto - F1-score: 81.56%


In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.model_selection import GridSearchCV

# Função para realizar predições com probabilidades
def kfoldcv_proba(hist, tags, k):
    fold_size = len(hist) // k
    all_probas = []
    y_tests = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Configurar os parâmetros para o GridSearchCV (SVM)
        param_grid = {
            'C': [0.289],
            'gamma': [1],
            'kernel': ['linear', 'rbf']
        }

        svm_model = SVC(random_state=42, probability=True)
        grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        
        # Melhor modelo encontrado pelo Grid Search
        best_model = grid_search.best_estimator_

        # Fazer previsões de probabilidades
        y_proba = best_model.predict_proba(X_test)
        all_probas.append(y_proba)
        y_tests.append(y_test)
    
    return np.concatenate(all_probas), np.concatenate(y_tests)

# Realizar a validação cruzada com probabilidades para cada conjunto de dados
proba_ori_svm, y_test = kfoldcv_proba(hist_ori, tags_ori, 3)
proba_har_svm, _ = kfoldcv_proba(hist_har, tags_har, 3)
proba_per_svm, _ = kfoldcv_proba(hist_per, tags_per, 3)

# Fusão das predições usando a regra da soma
fused_proba_soma = (proba_ori_svm + proba_har_svm + proba_per_svm) / 3
fused_pred_soma = np.argmax(fused_proba_soma, axis=1)
fused_pred_soma += 1  # Ajustar a classe para começar de 1

# Fusão das predições usando a regra do produto
fused_proba_produto = proba_ori_svm * proba_har_svm * proba_per_svm
fused_pred_produto = np.argmax(fused_proba_produto, axis=1)
fused_pred_produto += 1  # Ajustar a classe para começar de 1

# Avaliação das fusões
def evaluate_predictions(y_true, y_pred, method_name):
    accuracy = accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred, average='weighted') * 100
    recall = recall_score(y_true, y_pred, average='weighted') * 100
    f1 = f1_score(y_true, y_pred, average='weighted') * 100

    return accuracy, precision, recall, f1

# Função para realizar a avaliação com desvio padrão
def evaluate_with_std(y_true, y_pred, method_name):
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    for i in range(len(y_true)):
        accuracy, precision, recall, f1 = evaluate_predictions(y_true[i], y_pred[i], method_name)
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
    
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)
    
    mean_precision = np.mean(precisions)
    std_precision = np.std(precisions)
    
    mean_recall = np.mean(recalls)
    std_recall = np.std(recalls)
    
    mean_f1_score = np.mean(f1_scores)
    std_f1_score = np.std(f1_scores)

    print(f'{method_name} - Acurácia: {mean_accuracy:.2f}% ± {std_accuracy:.2f}%')
    print(f'{method_name} - Precisão: {mean_precision:.2f}% ± {std_precision:.2f}%')
    print(f'{method_name} - Recall: {mean_recall:.2f}% ± {std_recall:.2f}%')
    print(f'{method_name} - F1-score: {mean_f1_score:.2f}% ± {std_f1_score:.2f}%\n')

# Avaliar as predições usando a regra da soma
evaluate_with_std([y_test], [fused_pred_soma], "Regra da Soma")

print('----------------------------------------------------')

# Avaliar as predições usando a regra do produto
evaluate_with_std([y_test], [fused_pred_produto], "Regra do Produto")


Regra da Soma - Acurácia: 81.00% ± 0.00%
Regra da Soma - Precisão: 81.20% ± 0.00%
Regra da Soma - Recall: 81.00% ± 0.00%
Regra da Soma - F1-score: 80.99% ± 0.00%

----------------------------------------------------
Regra do Produto - Acurácia: 81.56% ± 0.00%
Regra do Produto - Precisão: 81.79% ± 0.00%
Regra do Produto - Recall: 81.56% ± 0.00%
Regra do Produto - F1-score: 81.56% ± 0.00%



In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np

# Função para realizar predições com probabilidades usando KNN
def kfoldcv_knn_proba(hist, tags, k):
    fold_size = len(hist) // k
    all_probas = []
    y_tests = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])

        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]

        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Definir a grade de parâmetros para o Grid Search do KNN
        param_grid = {
            'n_neighbors': [3, 5, 7, 9, 11],
            'metric': ['canberra', 'minkowski', 'euclidean', 'manhattan']
        }

        # Treinar o modelo KNN usando Grid Search
        knn_model = KNeighborsClassifier()
        grid_search = GridSearchCV(knn_model, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train, y_train)

        # Melhor modelo encontrado pelo Grid Search
        best_knn = grid_search.best_estimator_

        # Fazer previsões de probabilidades
        y_proba = best_knn.predict_proba(X_test)
        all_probas.append(y_proba)
        y_tests.append(y_test)

    return np.concatenate(all_probas), np.concatenate(y_tests)

# Realizar a validação cruzada com probabilidades para cada conjunto de dados
proba_ori_knn, y_test = kfoldcv_knn_proba(hist_ori, tags_ori, 3)
proba_har_knn, _ = kfoldcv_knn_proba(hist_har, tags_har, 3)
proba_per_knn, _ = kfoldcv_knn_proba(hist_per, tags_per, 3)

# Fusão das predições usando a regra da soma
fused_proba_soma = (proba_ori_knn + proba_har_knn + proba_per_knn) / 3
fused_pred_soma = np.argmax(fused_proba_soma, axis=1)

for i in range(0, len(fused_pred_soma)):
    fused_pred_soma[i] = fused_pred_soma[i] + 1

# Fusão das predições usando a regra do produto
fused_proba_produto = proba_ori_knn * proba_har_knn * proba_per_knn
fused_pred_produto = np.argmax(fused_proba_produto, axis=1)

for i in range(0, len(fused_pred_produto)):
    fused_pred_produto[i] = fused_pred_produto[i] + 1

# Função para avaliar as predições
def evaluate_predictions(y_true, y_pred, method_name):
    accuracy = accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred, average='weighted') * 100
    recall = recall_score(y_true, y_pred, average='weighted') * 100
    f1 = f1_score(y_true, y_pred, average='weighted') * 100

    print(f'{method_name} - Acurácia: {accuracy:.2f}%')
    print(f'{method_name} - Precisão: {precision:.2f}%')
    print(f'{method_name} - Recall: {recall:.2f}%')
    print(f'{method_name} - F1-score: {f1:.2f}%')

# Avaliar as predições usando a regra da soma
evaluate_predictions(y_test, fused_pred_soma, "Regra da Soma")

print('----------------------------------------------------')

# Avaliar as predições usando a regra do produto
evaluate_predictions(y_test, fused_pred_produto, "Regra do Produto")


Regra da Soma - Acurácia: 69.11%
Regra da Soma - Precisão: 69.36%
Regra da Soma - Recall: 69.11%
Regra da Soma - F1-score: 68.31%
----------------------------------------------------
Regra do Produto - Acurácia: 70.00%
Regra do Produto - Precisão: 70.85%
Regra do Produto - Recall: 70.00%
Regra do Produto - F1-score: 69.54%


In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Função para combinar probabilidades de KNN e SVM
def combine_probabilities(proba_knn, proba_svm):
    # Combina as probabilidades usando uma média simples
    combined_proba = (proba_knn + proba_svm) / 2
    return combined_proba

# Fusão das probabilidades dos modelos KNN e SVM
proba_ori_combined = combine_probabilities(proba_ori_knn, proba_ori_svm)
proba_har_combined = combine_probabilities(proba_har_knn, proba_har_svm)
proba_per_combined = combine_probabilities(proba_per_knn, proba_per_svm)

# Fusão das predições usando a regra da soma
fused_proba_soma = (proba_ori_combined + proba_har_combined + proba_per_combined) / 3
fused_pred_soma = np.argmax(fused_proba_soma, axis=1)

for i in range(0, len(fused_pred_soma)):
    fused_pred_soma[i] = fused_pred_soma[i] + 1

# Fusão das predições usando a regra do produto
fused_proba_produto = proba_ori_combined * proba_har_combined * proba_per_combined
fused_pred_produto = np.argmax(fused_proba_produto, axis=1)

for i in range(0, len(fused_pred_produto)):
    fused_pred_produto[i] = fused_pred_produto[i] + 1

# Função para avaliar as predições
def evaluate_predictions(y_true, y_pred, method_name):
    accuracy = accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred, average='weighted') * 100
    recall = recall_score(y_true, y_pred, average='weighted') * 100
    f1 = f1_score(y_true, y_pred, average='weighted') * 100

    print(f'{method_name} - Acurácia: {accuracy:.2f}%')
    print(f'{method_name} - Precisão: {precision:.2f}%')
    print(f'{method_name} - Recall: {recall:.2f}%')
    print(f'{method_name} - F1-score: {f1:.2f}%')

# Avaliar as predições usando a regra da soma
evaluate_predictions(y_test, fused_pred_soma, "Regra da Soma")

print('----------------------------------------------------')

# Avaliar as predições usando a regra do produto
evaluate_predictions(y_test, fused_pred_produto, "Regra do Produto")


Regra da Soma - Acurácia: 79.00%
Regra da Soma - Precisão: 79.06%
Regra da Soma - Recall: 79.00%
Regra da Soma - F1-score: 78.82%
----------------------------------------------------
Regra do Produto - Acurácia: 79.44%
Regra do Produto - Precisão: 79.59%
Regra do Produto - Recall: 79.44%
Regra do Produto - F1-score: 79.32%


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
import numpy as np

# Função para realizar a validação cruzada
def kfoldcv(hist, tags, k):
    fold_size = len(hist) // k
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Configurar os parâmetros para o GridSearchCV (SVM)
        param_grid = {
            'C': [0.289],
            'gamma': [1],
            'kernel': ['linear', 'rbf']
        }

        svm_model = SVC(random_state=42)
        grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        
        # Melhor modelo encontrado pelo Grid Search
        best_model = grid_search.best_estimator_

        # Fazer previsões e calcular as métricas
        y_pred = best_model.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred, average='weighted'))
        recalls.append(recall_score(y_test, y_pred, average='weighted'))
        f1_scores.append(f1_score(y_test, y_pred, average='weighted'))
    
    mean_accuracy = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy = np.std(accuracies) * 100  # Converter para porcentagem
    
    mean_precision = np.mean(precisions) * 100
    std_precision = np.std(precisions) * 100
    
    mean_recall = np.mean(recalls) * 100
    std_recall = np.std(recalls) * 100
    
    mean_f1_score = np.mean(f1_scores) * 100
    std_f1_score = np.std(f1_scores) * 100

    return (mean_accuracy, std_accuracy, 
            mean_precision, std_precision, 
            mean_recall, std_recall, 
            mean_f1_score, std_f1_score)

# Realizar a validação cruzada
(mean_accuracy, std_accuracy, 
 mean_precision, std_precision, 
 mean_recall, std_recall, 
 mean_f1_score, std_f1_score) = kfoldcv(hist, tags, 3)

print(f'Acurácia média: {mean_accuracy:.2f}% (Desvio padrão: {std_accuracy:.2f}%)')
print(f'Precisão média: {mean_precision:.2f}% (Desvio padrão: {std_precision:.2f}%)')
print(f'Recall médio: {mean_recall:.2f}% (Desvio padrão: {std_recall:.2f}%)')
print(f'F1-score médio: {mean_f1_score:.2f}% (Desvio padrão: {std_f1_score:.2f}%)')


In [8]:
import numpy as np

# Abrir o arquivo para escrita
arquivo = open('resultados.txt', 'w')

# Função para salvar as probabilidades em um formato legível
def save_probabilities(arquivo, probas, label):
    arquivo.write(f"{label}:\n")
    cont = 1
    for i, prob in enumerate(probas):
        prob_str = ", ".join([f"{p:.4f}" for p in prob])
        arquivo.write(f"tag {cont}: {prob_str}\n")
        
        if (i + 1) % 30 == 0:  # Incrementa o contador a cada 30 iterações
            cont += 1
        if i == 299:  # Reinicia o contador após 300 iterações
            cont = 1
    arquivo.write("\n")

# Salvar as probabilidades
save_probabilities(arquivo, proba_ori, "Probabilidades Originais")
save_probabilities(arquivo, proba_har, "Probabilidades Harmônicas")
save_probabilities(arquivo, proba_per, "Probabilidades Percussivas")

# Fechar o arquivo
arquivo.close()
